In [ ]:
import torch #importa libreria PyTorch per l'elaborazione del tensore
import torch.nn as nn # importa modulo nn , contiene le funzioni e le classi per la definizione di reti neurali

class Discriminator(nn.Module): # classe che eredita dalla classe nn.Module
    def __init__(self, channels_img, features_d):# metodo che  inizializza il discriminatore, con due parametri in ingresso: rispettivamente il numero di canali dell'immagine e il numero di features maps
        super(Discriminator,self).__init__()# chiama il costruttore della classe padre
        self.disc = nn.Sequential(
            # definisce una sequenza di strati neurali 'conteiner'
            # IN : N x channels_img x 64 x 64
            nn.Conv2d(

               # primo strato convoluzionale.
                #Prendi in ingresso immagini con il numero specificato da channels-img e rstituisce features map.
                #il filtro è 4x4, lo stride è 2 => l'output sarà di una dimensione minore.
                #padding= 1 (aggunge zeri intorno all'immagine di input)

                channels_img, features_d, kernel_size = 4, stride= 2, padding= 1
            ),# 32x32
            nn.LeakyReLU(0.5),

            # funzione di attivazione non lineare applicata alle feature map dello strato precedente.
             #La costante 0.5 specifica il coefficiente di pendanza per i valori negativi

            self._blok(features_d,features_d*2,4,2,1),# 16 x 16
            self._blok(features_d * 2, features_d * 4, 4, 2, 1),#8 x 8
            self._blok(features_d * 4, features_d * 8, 4, 2, 1),# 4 x 4
            nn.Conv2d(features_d * 8, 1, kernel_size=4,stride=2, padding=0),# restituisce un singolo valore in out
            nn.Sigmoid(),

             #funzione di attivazione per produrre un valore compreso tra 0 e 1,
             #rappresenta la probabilità che l'input sia un immagine reale(1)o immagine generata(0)

        )
        #metodo privato blok per creare strati del nostro modello
    def _blok(self, in_channels, out_channels,kernel_size, stride, padding):

      #Definisce un blocco di convoluzione con normalizzazione batch e LeakyReLU.
      #Prende come argomenti il numero di canali in ingresso,
      #il numero di canali in uscita, le dimensioni del kernel,
      #lo stride e il padding.

        return nn.Sequential( # restutuisce una sequenza di strati
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias = False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),# segue implementazione paper
        )
    def forward(self, x): # definisce come i dati vengono propagati attraverso il discriminatore
        return self.disc(x) # input passa attraverso il corpo prinzipale del discriminatore


class Generator(nn.Module):# eredita da nn.Module per crare il generatore
    def __init__(self, z_dim, channels_img, features_g):

    #inizializza il generatore con: dimensione dello spazio latente,
    #numero canali dell'immagine in output e il numero di feature maps desiderate

        super(Generator,self).__init__()# chiama il metodo di nìinizializzazione della calssa genitor, per inizializzare correttamente la classe generetor
        self.gen = nn.Sequential( # definisce il generatore con i diversi strati, corpo principale del modello
            #IN: N x z_dim x 1 x 1
            self._blok( z_dim, features_g * 16,4,1,0),#N x f_g*16 x 4 x 4
            self._blok(features_g * 16, features_g * 8, 4, 2, 1),# 8x8
            self._blok(features_g * 8, features_g * 4, 4, 2, 1),#16x16
            self._blok(features_g * 4, features_g * 2, 4, 2, 1),#32x32
            nn.ConvTranspose2d(features_g * 2, channels_img, kernel_size=4, stride=2, padding=1),

            #prende le feature map dallo strato precedente
            #e produce un'immagine con il numero di canali specificato da ' channels-img'

            nn.Tanh(), #funzione di attivazione per garantire che i valori siano compresi nell'intervallo [-1, 1]
        )
    def _blok(self, in_channels, out_channels,kernel_size, stride, padding):

      #metodo definisce un blocco di convoluzione trasposta con normalizzazione batch e ReLU.
      #Prende come argomenti il numero di canali in ingresso,
      #il numero di canali in uscita, le dimensioni del kernel,
      #lo stride e il padding.

        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias = False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def forward(self, x): # come vengono propagati i dati attraverso il generatore
        return self.gen(x)

def initialize_weights(model):# inizializzazione dei pesi dei modelli

#prende come input un modello di rete neurale (model).
#Itera attraverso tutti i moduli del modello (model.modules()) e,
#se il modulo è un'istanza di nn.Conv2d, nn.ConvTranspose2d, o nn.BatchNorm2d,
#inizializza i pesi di quel modulo con una distribuzione normale con media 0.0 e deviazione standard 0.02.

    for n in model.modules():
        if isinstance(n, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(n.weight.data, 0.0, 0.02)

def test():# definisce una serie di test
    N, in_channels, H, W = 8, 3, 64, 64
    z_dim = 100
    x = torch.randn(N, in_channels, H, W)# batch di dati casuali
    disc = Discriminator(in_channels, 8)# istanziato modello
    initialize_weights(disc)
    assert disc(x).shape == (N,1,1,1)# contollo per verificare se le dimensioni delle uscite sono corrette
    gen = Generator(z_dim, in_channels, 8)
    initialize_weights(gen)
    z = torch.randn((N, z_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W)
    print('Success')

test()

Success


In [ ]:
import zipfile # modulo per lavorare con dile ZIP
import os # modulo per lavorare xon operazioni di sistema operativo

# Percorso del file ZIP sul Google Drive
zip_path = "/content/drive/MyDrive/Alessandro Rossomando/img_align_celeba.zip"

# Percorso di destinazione per l'estrazione
extract_path = "/content/img_align_celeba"


with zipfile.ZipFile(zip_path, 'r') as zip_ref:# estrae il contenuto dle file ZIP (zip_path)
    zip_ref.extractall(extract_path) # estrae tutto il contenuto per inserirlo nella destinazione specificata(extract_path)

# Verifica l'estrazione : restituisce una lista dei file e delle cartelle presenti nella cartella estratta
os.listdir(extract_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Alessandro Rossomando/img_align_celeba.zip'

In [ ]:
import torch #mpdulo PyTorch per funzionalità di base
import torch.nn as nn # per definire le reti neurali
import torch.optim as optim # per  gli ottimizzatori
import torchvision # getire dei dataset di immagini
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import matplotlib.pyplot as plt # visualizzazione delle immgini
import matplotlib.image as mpimg # per il caricamento delle immagini
genlosses = []
dislosses = []

# Parametri
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # dispositivo di esecuzione
IMAGE_SIZE = 64
CHANNELS_IMG = 3

# Definizione delle trasformazioni
transforms = transforms.Compose([

    #definisce le trasformazioni da applicare alle immagini prima di passarle al modello
    #le immagini vengono ridimensionate,convertite in tensori e normalizzate

    transforms.Resize(IMAGE_SIZE),
    #transforms.CenterCrop(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5 for _ in range(CHANNELS_IMG)] ,[0.5 for _ in range(CHANNELS_IMG)]),
])
#carico il dataset dal percorso specificato e applica le trasformazioni
dataset_full = datasets.ImageFolder(root="img_align_celeba",transform = transforms )
num_img = 3500

subset_idx = list(range(num_img))
subset_sampler = SubsetRandomSampler(subset_idx)# creo un campione casuale di immagini dal dataset
dataset_subset = torch.utils.data.Subset(dataset_full, subset_idx)

# Def diverse combinazioni di iperparametri da provare
hyperparameters = [
   # {"L_R": 0.0001, "BATCH_SIZE": 64, "Z_DIM": 64, "NUM_EPOCHS": 10, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
    {"L_R": 0.0001, "BATCH_SIZE": 64, "Z_DIM": 64, "NUM_EPOCHS": 3, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
   # {"L_R": 0.0005, "BATCH_SIZE": 64, "Z_DIM": 64, "NUM_EPOCHS": 10, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
    #{"L_R": 0.0005, "BATCH_SIZE": 64, "Z_DIM": 64, "NUM_EPOCHS": 30, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
    #{"L_R": 0.0001, "BATCH_SIZE": 128, "Z_DIM": 64, "NUM_EPOCHS": 10, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
    #{"L_R": 0.0001, "BATCH_SIZE": 128, "Z_DIM": 64, "NUM_EPOCHS": 30, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
    #{"L_R": 0.0005, "BATCH_SIZE": 128, "Z_DIM": 64, "NUM_EPOCHS": 10, "FEATURES_DISC": 32, "FEATURES_GEN": 32},
    #{"L_R": 0.0005, "BATCH_SIZE": 128, "Z_DIM": 64, "NUM_EPOCHS": 30, "FEATURES_DISC": 32, "FEATURES_GEN": 32},

     #{"L_R": 0.0002, "BATCH_SIZE": 128, "Z_DIM": 128, "NUM_EPOCHS": 100, "FEATURES_DISC": 64, "FEATURES_GEN": 64},
    # Aggiungi altre combinazioni di iperparametri che desideri provare
]

for idx, params in enumerate(hyperparameters):
     # Estrai i parametri dalla configurazione corrente
     L_R = params["L_R"]
     BATCH_SIZE = params["BATCH_SIZE"]
     Z_DIM = params["Z_DIM"]
     NUM_EPOCHS = params["NUM_EPOCHS"]
     FEATURES_DISC = params["FEATURES_DISC"]
     FEATURES_GEN = params["FEATURES_GEN"]


     loader = DataLoader ( dataset_subset ,batch_size= BATCH_SIZE ,shuffle=True)# DataLoader per caricare i dati in batch durante l'addestramento del modello
     gen = Generator(Z_DIM,CHANNELS_IMG,FEATURES_GEN,).to(device)# inizializza generatore e li sposta sul dispositivo specificato
     disc = Discriminator(CHANNELS_IMG,FEATURES_DISC).to(device)# inizializza discriminatore
     initialize_weights(gen)# inizializza pesi del modello
     initialize_weights(disc)

    #definisco gli ottimizzatori e la funzione di loss
     opt_gen = optim.NAdam(gen.parameters(),lr = L_R, betas=(0.5,0.999))
     opt_disc = optim.NAdam(disc.parameters(),lr = L_R, betas=(0.5,0.999))
     criterion = nn.BCELoss() # Binary Cross Entropy Loss

     fixed_noise= torch.randn(32,Z_DIM,1,1).to(device) # tensore di rumore fisso per generare immagini durante il training
     step = 0

    # imposto i modelli in modalità di addestramento
     gen.train()
     disc.train()

    #ciclo principale di addestramento
    #scorre le epoche e le iterazioni all'interno di ciascuna epoca, ottenendo i dati dal 'DataLoader'
     for epoch in range(NUM_EPOCHS):
         for batch_idx, (real, _) in enumerate(loader):

             real = real.to(device)# vengono spostati i dati sul dispositivo
             noise = torch.randn((BATCH_SIZE, Z_DIM,1,1)).to(device)# generato il rumore per le immagini finte
             fake = gen(noise)# generata un'immagine finta utilizzando il modello

            ### Train Discriminator max log(D(x)) + log(1- D(G(z)))
             disc_real= disc(real).reshape(-1)
             loss_disc_real=criterion(disc_real, torch.ones_like(disc_real))
             disc_fake = disc(fake).reshape(-1)
             loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
             loss_disc = (loss_disc_real + loss_disc_fake)/ 2
             disc.zero_grad()
             torch.cuda.empty_cache()
             dislosses.append(loss_disc)
             loss_disc.backward(retain_graph = True)
             opt_disc.step()
          # torch.cuda.empty_cache()

            ### Train Generator min log(1-D(g(z))) === max log(D(G(z))
             output = disc(fake).reshape(-1)
             loss_gen = criterion(output, torch.ones_like(output))
             genlosses.append(loss_gen)
             gen.zero_grad()
             loss_gen.backward()
             opt_gen.step()

            #torch.cuda.empty_cache()

            # Print losses occasionally and print
             if batch_idx % 30 == 0:

              #ogni 50 iterazioni viene stampato il valore della loss per entambi i modelli
                 print(
                     f"[Epoch {epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)} \
                       D Loss: {loss_disc:.6f}, G Loss: {loss_gen:.6f}"
                 )
                 with torch.no_grad():
                     fake = gen(fixed_noise)


                    # Visualizza un campione di immagini generate
                     fig, axes = plt.subplots(3, 3, figsize=(5, 5))
                     for i, ax in enumerate(axes.flatten()):
                         img_normalized = (fake[i].cpu().permute(1, 2, 0).numpy() + 1) / 2
                         ax.imshow(img_normalized)
                         ax.axis('off')
                     plt.show()
                 step += 1
                 print(f"Configurazione {idx+1}: Loss generatore = {loss_gen:.6f},   D Loss: {loss_disc:.6f}")





In [ ]:
ge = loss_gen.clone().detach().cpu().long()
de = loss_disc.clone().detach().cpu().long()



plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(ge,label="G")
plt.plot(de,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()